In [1]:
from mne_bids_pipeline._config_import import _import_config
import mne
cfg = _import_config(
        config_path='./mne-bids/config/mne-bids-pipeline.py'
    )
power = mne.time_frequency.read_tfrs('../data/ds003702/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_power+s3042+tfr.h5', condition=0) # TODO: should probably get the path from the config too
power.apply_baseline(
    baseline=cfg.time_frequency_baseline,
    mode=cfg.time_frequency_baseline_mode,
)
if cfg.time_frequency_crop:
    power.crop(**cfg.time_frequency_crop)
%matplotlib qt

In [4]:
fig_power = power.plot_topomap(ch_type='eeg', show=True)
fig_power.axes[0].set_title('Alpha: Stick')
fig_power.set_size_inches(5, 5)

fig_power.savefig("alpha_stick.pdf")

In [3]:
fig_power = power.plot_topo(title='Alpha: Stick', fig_facecolor="w", font_color="k", border="k", show=True)
fig_power.set_size_inches(9, 6)

In [6]:
from visualize import power

power.plotPower('./mne-bids/config/mne-bids-pipeline.py', subject='01', 
                cue='sticks', band='theta', time_range=(3.6, 4.0),
                filename='power-{cue}-{band}-{tmin}-to-{tmax}.png')

power.plotPower('./mne-bids/config/mne-bids-pipeline.py', subject='01',
                cue='avatar', band='theta', time_range=(3.6, 4.0),
                filename='power-{cue}-{band}-{tmin}-to-{tmax}.png')


power.plotPower('./mne-bids/config/mne-bids-pipeline.py', subject='01',
                cue='sticks', band='theta', time_range=(2.0, 3.0),
                filename='power-{cue}-{band}-{tmin}-to-{tmax}.png')

power.plotPower('./mne-bids/config/mne-bids-pipeline.py', subject='01',
                cue='avatar', band='theta', time_range=(2.0, 3.0),
                filename='power-{cue}-{band}-{tmin}-to-{tmax}.png')